# Creating the synthetic spectra.

## 0 Packages and data

In [24]:
path='G:/Shared drives/BeStarsMiMeS/UpdatedFiles/' #Patrick's google file stream path

In [4]:
import specpolFlow as pol
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
import os
import copy
import itertools
import pandas as pd

loading specpolFlow package


In [5]:
import pandas as pd
sheet_id = '1M6y1Wnsrc-w5FjUMfKaSFa_-foIDAaMe8W4lYNWnWyk'
sheet_name = 'Stars'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
StarData=pd.read_csv(url)

sheet_name = 'Observations'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
Observations=pd.read_csv(url)

## 1. Calculating the disk integrated, rotationally broadened spectra

### 1.1 Demo with a single observation

A succesfull calculation should output:

```
 Integration done
```
and create a new file in the `Synth-diskint-spectra` folder on the Shared Google Drive

There might be this error as well
```
ERROR: ld.so: object '/usr/lib/x86_64-linux-gnu/libtcmalloc.so.4' from LD_PRELOAD cannot be preloaded (wrong ELF class: ELFCLASS64): ignored.
```
but it does not seem to affect the calculation.

In [ ]:
# Need to change the permissions on the executable
# So that it will run.
! chmod 755 /01-Synth-calculations/Synth-codes/s3div.Linux

In [6]:
star = 'hd110432'
ModelCode='T27000G35'
vsini = 230.0
vmac = 2.0
res = 65000


file_mout = '{}00-InputMaterial/Synth-local-spectra/{}.mout'.format(path,ModelCode)
file_int  = '{}01-Synth-calculations/Synth-diskint-spectra/{}.dsk'.format(path,star)

command = '{}01-Synth-calculations/Synth-codes/s3div.Linux {} {} {} {} 167000 {}'.format(path,file_mout, file_int, vsini, vmac, res)
print(command)
stream = os.popen(command)
output = stream.readlines()
output


01-Synth-calculations/Synth-codes/s3div.Linux G:/Shared drives/BeStarMiMeS/00-InputMaterial/Synth-local-spectra/T27000G35.mout 01-Synth-calculations/Synth-diskint-spectra/hd110432.dsk 230.0 2.0 167000 65000


[]

### 1.2 Loop over all the stars. 

> **TODO**: make a loop that makes the calculation above for each star, using the information in the 00-Information spreadsheets. 

In [ ]:
#Code by Marisol Catalan Olais, updated by Patrick Stanley

#Star Data is the Stars worksheet

for i in range(len(StarData)):
  star_name = StarData["Name"][i]
  model_code = StarData["ModelCode"][i].upper()
  if StarData['vsini-estimate'][i]=='very bad':
    adopt_vsini = StarData["Adopted-vsini"][i]
  else:
    adopt_vsini = StarData['vsini-estimate'][i]
  vmac = 2.0
  res = 65000

  file_mout = '{}00-InputMaterial/Synth-local-spectra/{}.mout'.format(path,model_code)
  file_int  = '{}01-Synth-calculations/Synth-diskint-spectra/{}.dsk'.format(path,star_name)

  command = '{}01-Synth-calculations/Synth-codes/s3div.Linux {} {} {} {} 167000 {}'.format(path,file_mout, file_int, adopt_vsini, vmac, res)
  print(command)
  stream = os.popen(command)
  output = stream.readlines()
  
  print(output)

## 2. Adding radial velocity shift and noise. 

Gregg originally had an IDL code to do this, using a list of SNR (from the headers of the observations?)

But as we do have the original spectra handy, I propose a maybe simpler python procedure. 

1. Read in the original spectrum for an observation
2. Read in the rotationally broadened model spectrum that has been convolved with the instrument resolution by `s3div` already. 
3. Shift the model by the appropriate radial velocity for that observation.
4. Split the observation into its orders (so this is assuming that the spectrum's orders have not been merged. If using a code that merges the order, then there should be an option to do this procedure in one swoop, instead of order by order. For each order, do the folloowing:
  - a. Interpolate the shifted model spectrum to the wavelength grid of the observed spectrum
  - b. Add some random noise to the interpolated model, using the observed error column as the sigma of the normal distribution. 
  - c. Copy over all of the other columns from the data -- this way there is no need to stitch together the LSD profiles after the fact. 
5. Save the resulting spectrum in .s format. 

Functions that will be used below (move this to a module on the github, maybe? split and splice could go into the iolsd.observation class, once there is a check added in case there are no spectral orders present -- i.e. for already mearged observations)

In [20]:
def read_disk(file):
  '''
  Read a synthetic spectrum from a s3div output

  :param file: The filename
  :rtype: wave, flux -- the wavelength in nm, and the normalized flux. 
  '''
  with open(file) as f:
    lines = f.readlines()
  # number of atomic lines in the header of the file
  natomic = int(lines[0].split(sep='-')[0])
  nwave=int(lines[natomic+1].split()[0])
  data = np.genfromtxt(lines[natomic+2:natomic+1+nwave])
  return(data[:,0]/10.0, data[:,1])

def split_order(data):
  '''
  Split an observation object into a list of observation objects with one order per item
  '''
  # one order is where the wavelength backtracks. 
  ind = np.where((data.wl[1:]-data.wl[0:-1]) < 0)[0]
  norder = ind.size+1
  ind = np.append(-1,ind)
  ind = np.append(ind,data.wl.size)
  print('{} orders'.format(norder))

  list_order=[]
  for i in range(0,norder):
    list_order.append(data[ind[i]+1:ind[i+1]])
 
  return(list_order)
  
def splice_order(list_order):
  '''
  Concatenate a list of observation objects (usually split into a list of orders by split_order function). Note: it does not merge the orders overlaps.
  '''
  wl = np.array([])
  specI = np.array([])
  specV = np.array([])
  specN1 = np.array([])
  specN2 = np.array([])
  specSig = np.array([])
  for item in list_order:
    wl=np.append(wl,item.wl)
    specI=np.append(specI,item.specI)
    specV=np.append(specV,item.specV)
    specN1=np.append(specN1,item.specN1)
    specN2=np.append(specN2,item.specN2)
    specSig=np.append(specSig, item.specSig)
  return(pol.Spectrum(wl, specI, specV, specN1, specN2, specSig, header=list_order[0].header))

def rshift(wave, radvel):
  '''
  Shift a spectrum in terms of radial velocity. The wavelength array and the velocity arrays must be defied as astropy units quantities.
  The new wavelength array is returned in the same units as the initial wavelength array. 
  '''
  return( (wave + wave*radvel/const.c).to(wave.unit).value )


### 2.1 Demo with a single observation

In [25]:
star = 'hd110432'
obs = '1'
vradCorrected = -5.4656055

> **TODO** Need to check whether this will work with normalized HARPS spectra (seem to remember something funny in the sorting of the wavelength arrays...)
(star #3 on the list has a HARPS spectrum, I think)

In [26]:
# reading the observed spectrum
file_obs = '{}00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(path,star, obs)
data_obs = pol.read_spectrum(file_obs)
# splitting the observed spectrum by order
list_order = split_order(data_obs)

# reading the model spectrum
file_mod = '{}01-Synth-calculations/Synth-diskint-spectra/{}.dsk'.format(path,star)
mod_wave, mod_flux = read_disk(file_mod)

# shifting the model spectrum for its radial velocity
# (note the rshift function asks for numpy unit quantities)
mod_wave_shift = rshift(mod_wave*u.nm, vradCorrected*u.km/u.s)

for order in list_order:

  # interpolate the shifted model onto the order grid
  mod_flux_interp = np.interp(order.wl, mod_wave_shift, mod_flux)

  # drawing n random elements with a mean of zero and a stdev of 1.0
  # scaling to a sigma corresponding to the error bar
  noise = np.random.standard_normal(order.wl.size)*order.specSig
  mod_flux_interp = mod_flux_interp + noise
  # replacing the flux array in the order
  order.specI = copy.deepcopy(mod_flux_interp)

# putting the orders back together
new = splice_order(list_order)

### writting back the file in the proper folder. 
## need to look for a write function for .s, maybe in Colin's 
## normalization code?

new.save('{}01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(path,star,obs))

70 orders


Quick graph to check the result -- the graph is not needed for the loop below. The comparison between model and data is done in the 03-NormalizedSpectra notebook. 

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,4))
ax.plot(data_obs.wl, data_obs.specI)
ax.plot(mod_wave, mod_flux)
ax.plot(new.wl, new.specI)
ax.set_xlim(400,405)
ax.set_ylim(0.5,1.1)

### 2.2 Loop over all the observations

>**TODO**: make a loop over all of the observations to make the calculations above for each observation, using the information from the tables in 00-Information Google spreadsheet.

In [ ]:
#Code by Marisol Catalan Olais and Dax Moraes

for j in range(len(Observations)):
  Star = Observations["Name"][j]
  StarName = Observations["NameAsif"][j]
  
  ObsInfo = Observations.loc[Observations["NameAsif"] == StarName]
  #Vrad = float(ObsInfo["vradCorrected"])
  Vrad = float(ObsInfo["Unnamed: 11"])
  #Loc_vsini = StarData.loc[StarData["Name"] == Star]
  #AdoptVsini = float(Loc_vsini["Adopted-vsini"])

  file_obs = '{}00-InputMaterial/NormalizedSpectra/{}.s'.format(path,StarName) 
  data_obs = pol.read_spectrum(file_obs)
  list_order = split_order(data_obs)

  file_mod = '{}01-Synth-calculations/Synth-diskint-spectra/{}.dsk'.format(path,Star)
  mod_wave, mod_flux = read_disk(file_mod)

  mod_wave_shift = rshift(mod_wave*u.nm, Vrad*u.km/u.s)

  for order in list_order:
    mod_flux_interp = np.interp(order.wl, mod_wave_shift, mod_flux)
    
    noise = np.random.standard_normal(order.wl.size)*order.specSig
    mod_flux_interp = mod_flux_interp + noise
    
    order.specI = copy.deepcopy(mod_flux_interp)

  new = splice_order(list_order)

  new.save('{}01-Synth-calculations/Synth-hybrid-spectra/{}_noise.s'.format(path,StarName))

  fig, ax = plt.subplots(1,1, figsize=(12,4))
  ax.plot(data_obs.wl, data_obs.specI)
  ax.plot(mod_wave, mod_flux)
  ax.plot(new.wl, new.specI)
  ax.set_xlim(400,405)
  ax.set_ylim(0.5,1.1)
  ax.set_title(StarName)
  plt.rcParams.update({'figure.max_open_warning': 0})